In [2]:
# Added cell to set Working Directory to your location
import os
import re
import glob
import numpy as np
import pandas as pd
import json
import time

In [2]:
os.chdir("//Volumes/SSD/00_Data_Processing/Test_Exploration")
os.getcwd()

'/Volumes/SSD/00_Data_Processing/Test_Exploration'

In [3]:
# Columns to keep from the raw data.
Keep = ["SubjectID", "Session", "SessionSubsection",'timeStampDataPointStart', 'timeStampDataPointEnd', 'timeStampGetVerboseData','combinedGazeValidityBitmask','rayCastHitsCombinedEyes','eyePositionCombinedWorld.x', 'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z','eyeDirectionCombinedWorld.y', 
 'eyeDirectionCombinedWorld.z', 'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y', 'eyeDirectionCombinedLocal.z','hmdPosition.x', 'hmdPosition.y', 'hmdPosition.z', 'hmdDirectionForward.x', 'hmdDirectionForward.y', 'hmdDirectionForward.z', 
 'hmdRotation.x', 'hmdRotation.y', 'hmdRotation.z', 'hmdDirectionUp.x', 'hmdDirectionUp.y', 'hmdDirectionUp.z','playerBodyPosition.x', 'playerBodyPosition.y', 'playerBodyPosition.z', 'bodyTrackerPosition.x', 'bodyTrackerPosition.y', 
 'bodyTrackerPosition.z', 'bodyTrackerRotation.x', 'bodyTrackerRotation.y', 'bodyTrackerRotation.z']

In [4]:
data_path = '/Volumes/SSD/00_Data_Processing/Test_Exploration'
processed_data = '/Volumes/SSD/00_Data_Processing/Pre_processed'

# Getting the Folder without hidden files in ascending order
DATA_FOLDER = sorted([f for f in os.listdir(data_path) if not f.startswith('.')], key=str.lower)
PROCESSED_DATA_FOLDER = sorted([f for f in os.listdir(processed_data) if not f.startswith('.')], key=str.lower)
subIDs = []
for sub in DATA_FOLDER:
    if sub[0:4].isdigit():
        subIDs.append(int(sub[0:4]))
    else:
        pass
subIDs = np.unique(subIDs)
#Sincesome participant IDs start with 0, we format them to show it in the string type
IDstrings = ['{:04d}'.format(id) for id in subIDs]
print(IDstrings)

['0479', '1754', '2258', '2361', '2693', '3246', '3310', '3572', '3976', '4176', '4796', '4917', '5238', '5531', '5741', '6642', '7093', '7264', '7412', '7842', '8007', '8469', '8673', '8695', '9472', '9502', '9601']


In [7]:
#Create a generalized path for all json files per participant
paths = [ID +  "/*.json" for ID in IDstrings]
print(paths)
#Create a sorted list of the paths to open de jsons
Sorted_individual_jsons = sorted([filename for path in paths for filename in glob.glob(path)], key=str.lower)

['1754/*.json', '2258/*.json', '2361/*.json', '2693/*.json', '3246/*.json', '3310/*.json', '3572/*.json', '3976/*.json', '4176/*.json', '4796/*.json', '4917/*.json', '5238/*.json', '5531/*.json', '5741/*.json', '6642/*.json', '7093/*.json', '7264/*.json', '7412/*.json', '7842/*.json', '8007/*.json', '8469/*.json', '8673/*.json', '8695/*.json', '9472/*.json', '9502/*.json', '9601/*.json']


In [ ]:
data_raw = pd.DataFrame()

# read every file name in folder
for path in paths:
    for filename in glob.glob(path):
        with open(filename, 'r') as file:
            try:
                # make json files parsable
                data = "[" + file.read()
                data = data[:len(data)] + "]"
                raw = json.loads(data)
            except:
                print("reading did not work")
                

            # Uneast the higher level of each file
            currentDF_raw = pd.json_normalize(raw[0]['trials'][0]['dataPoints'])
            print( "Subject " + str(filename[5:9]) + " Session " + str(filename[17:19]) +" Section " + str(filename[23:24]) + " has been normalized")
            #Reduce columns to just necessary information
            print('time is: ', time.ctime())

            # insert participant id and session information from the file name
            currentDF_raw.insert(0, "SubjectID", [int(filename[5:9])] * currentDF_raw.shape[0], True)
            currentDF_raw.insert(1, "Session", [int(filename[17:19])] * currentDF_raw.shape[0], True)
            currentDF_raw.insert(2, "SessionSubsection", [int(filename[23:24])] * currentDF_raw.shape[0], True)
            
            #Take out the unnecesary information
            Reduced= currentDF_raw[Keep]

            #Normalize the collider hits
            Raycast = Reduced['rayCastHitsCombinedEyes'].explode().apply(pd.Series)
            Combined = pd.concat([Reduced, Raycast], axis=1)
            print('Combined')

        data_raw = data_raw.append(Combined, ignore_index=True)
        print('Appended')
    data_raw.sort_values(by=["SubjectID","Session","SessionSubsection"], inplace=True)
    data_raw.to_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/01_Indivuduals_Flat_smaller/" + str(filename[5:9]) + ".csv")
    print('Saved')
    data_raw = pd.DataFrame()

In [3]:
filenames = glob.glob("/Volumes/SSD/00_Data_Processing/Pre_processed/01_Indivuduals_Flat_smaller/*.csv")                    
combined_csv = pd.concat( [ pd.read_csv(f) for f in filenames ])

In [5]:
combined_csv.tail()

,Unnamed: 0,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,eyePositionCombinedWorld.x,...,bodyTrackerPosition.z,bodyTrackerRotation.x,bodyTrackerRotation.y,bodyTrackerRotation.z,0,hitColliderType,hitObjectColliderBoundsCenter,hitObjectColliderName,hitPointOnObject,ordinalOfHit
1897596,1897596,9601,5,3,1.652798e+09,1.652798e+09,1.652798e+09,3,[{'hitPointOnObject': {'x': -295.2583618164062...,-111.037956,...,-0.122341,331.750519,255.437515,236.807922,NaN,UnityEngine.MeshCollider,"{'x': -310.80328369140625, 'y': 3.311013460159...",TaskBuilding_8,"{'x': -307.9334716796875, 'y': 5.1120595932006...",2.0
1897597,1897597,9601,5,3,1.652798e+09,1.652798e+09,1.652798e+09,3,[{'hitPointOnObject': {'x': -276.6176147460937...,-111.081200,...,-0.122575,331.776093,254.747726,236.799957,NaN,UnityEngine.MeshCollider,"{'x': -291.20001220703125, 'y': 4.654793739318...",Building_213,"{'x': -276.61761474609375, 'y': 5.832995891571...",1.0
1897598,1897598,9601,5,3,1.652798e+09,1.652798e+09,1.652798e+09,3,[{'hitPointOnObject': {'x': -276.6176147460937...,-111.081200,...,-0.122575,331.776093,254.747726,236.799957,NaN,UnityEngine.MeshCollider,"{'x': -304.0008239746094, 'y': 11.011278152465...",Building_210,"{'x': -300.0303955078125, 'y': 6.0868353843688...",2.0
1897599,1897599,9601,5,3,1.652798e+09,1.652798e+09,1.652798e+09,3,[{'hitPointOnObject': {'x': -276.6176147460937...,-111.081200,...,-0.122575,331.776093,254.747726,236.799957,NaN,UnityEngine.MeshCollider,"{'x': -291.20001220703125, 'y': 4.654793739318...",Building_213,"{'x': -276.61761474609375, 'y': 5.832995891571...",1.0
1897600,1897600,9601,5,3,1.652798e+09,1.652798e+09,1.652798e+09,3,[{'hitPointOnObject': {'x': -276.6176147460937...,-111.081200,...,-0.122575,331.776093,254.747726,236.799957,NaN,UnityEngine.MeshCollider,"{'x': -304.0008239746094, 'y': 11.011278152465...",Building_210,"{'x': -300.0303955078125, 'y': 6.0868353843688...",2.0


In [7]:
combined_csv.to_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/Experimen1R.csv")

In [ ]:
data_raw.rayCastHitsCombinedEyes[0]

In [ ]:
data_raw.tail(25)

In [ ]:
reduced_data